In [152]:
from eventhandler import EventHandler
from datetime import datetime

from pprint import pprint


class StrategyBase:

    def __init__(self):
        self.event_handler = EventHandler('newSignal')
        self.event_handler.link(self.__on_new_signal, 'newSignal')

    def __on_new_signal(self, signal):
        print("=======[DEBUG]=========\n"
              f"StrategyBase.__on_new_signal fired!\n"
              "=======================\n")

    def emulate_signal(self, signal):
        self.event_handler.fire('newSignal', signal)


In [157]:
class PositionManager:

    def __init__(self, strategy: StrategyBase):
        self.current_position = None
        self.event_handler = EventHandler('onOpenPosition', 'onClosePosition')

        self.event_handler.link(self.__on_open_position, 'onOpenPosition')
        self.event_handler.link(self.__on_close_position, 'onClosePosition')


        strategy.event_handler.link(self.open, 'newSignal')
        strategy.event_handler.link(self.close, 'newSignal')

    # This callback will be called when onOpenPosition event happens
    def __on_open_position(self, position):
        print("=======[DEBUG]=========\n"
              "PositionManager.__on_open_position fired!\n")
        print(f"Position Opened:\n"
              f"{position}\n"
              "=======================\n")
        self.current_position = position

    # This callback will be called when onClosePosition event happens
    def __on_close_position(self, position):
        print("=======[DEBUG]=========\n"
              "PositionManager.__on_close_position fired!\n")
        print(f"Position Closed:\n"
              f"{position}"
              "=======================\n")

    # Now let's define the public methods of the PositionManager to be used outside the class
    def open(self, direction):
        print("=======[DEBUG]=========\n"
              "PositionManager.open() called\n")

        if direction != 'CLOSE':
            position = {
                "direction": direction,
                "open_timestamp": datetime.utcnow().timestamp(),
                "status": "open"
            }
            print(f"Opening {direction} position:\n")
            pprint(position)
            print("\nAbout to fire:\n"
                  "self.event_handler.fire('onOpenPosition', position)\n"
                  "=======================\n")

            self.event_handler.fire('onOpenPosition', position)
        else:
            print("signal was CLOSE, so not opening\n"
                  "=======================\n")

    def close(self, signal):
        print("=======[DEBUG]=========\n"
              "PositionManager.close() called.\n")
        if signal == 'CLOSE':
            position = self.current_position

            pprint(position)

            if type(position) == dict:

                print("=======[DEBUG]=========\n"
                      "PositionManager.open() called\n"
                      f"Closing Position:\n"
                      f"{position}")
                position['status'] = "closed"
                position['close_timestamp'] = datetime.utcnow().timestamp()
                print("About to fire:\n"
                      "self.event_handler.fire('onClosePosition', position)\n")
                self.event_handler.fire('onClosePosition', position)
        else:
            print("signal was not CLOSE, so not closing\n"
                  "=======================\n")


In [158]:
class Notifier:

    def __init__(self, pm: PositionManager):

        # self.event_handler = EventHandler()

        pm.event_handler.link(self.send_telegram_notification, 'onOpenPosition')
        pm.event_handler.link(self.send_telegram_notification, 'onClosePosition')

    def send_telegram_notification(self, position):
        print("Sending Telegram Notification...\n"
              "Position Info:\n")
        pprint(position)


In [159]:
my_strategy = StrategyBase()  # Must init with ()
# pprint(dir(my_strategy))
my_pm = PositionManager(my_strategy)

my_notifier = Notifier(my_pm)

my_strategy.emulate_signal('LONG')

=======[DEBUG]=========
StrategyBase.__on_new_signal fired!

=======[DEBUG]=========
PositionManager.open() called

Opening LONG position:

{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'open'}

About to fire:
self.event_handler.fire('onOpenPosition', position)

=======[DEBUG]=========
PositionManager.__on_open_position fired!

Position Opened:
{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'open'}

Sending Telegram Notification...
Position Info:

{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'open'}
=======[DEBUG]=========
PositionManager.close() called.

signal was not CLOSE, so not closing



In [160]:
my_strategy.emulate_signal('CLOSE')

StrategyBase.__on_new_signal fired!

PositionManager.open() called

signal was CLOSE, so not opening

PositionManager.close() called.

{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'open'}
PositionManager.open() called
Closing Position:
{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'open'}
About to fire:
self.event_handler.fire('onClosePosition', position)

PositionManager.__on_close_position fired!

Position Closed:
{'direction': 'LONG', 'open_timestamp': 1648332854.739824, 'status': 'closed', 'close_timestamp': 1648332855.357236}=======================

Sending Telegram Notification...
Position Info:

{'close_timestamp': 1648332855.357236,
 'direction': 'LONG',
 'open_timestamp': 1648332854.739824,
 'status': 'closed'}
